# Custom ZnTrackOptions

ZnTrack allows you to create a custom ZnTrackOption similar to `zn.outs`.
ZnTrack tries to handle some standard types automatically within the `zn.outs` option, but it can be useful to write custom ones.
In the following example we use [Atomic Simulation Environment](https://wiki.fysik.dtu.dk/ase/index.html) to store / load objects to a custom datafile.

In [1]:
from zntrack import config

# When using ZnTrack we can write our code inside a Jupyter notebook.
# We can make use of this functionality by setting the `nb_name` config as follows:
config.nb_name = "08_custom_zntrackoptions.ipynb"

In [2]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

In [3]:
!git init
!dvc init

Initialized empty Git repository in /tmp/tmpcevd90om/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


We will use the `ZnTrackOption` to build our new custom options.

In [4]:
import zntrack
import ase.db
import ase.io
import tqdm
import pathlib

In [22]:
class Atoms(zntrack.Field):
    # we will save the file as dvc run --outs
    dvc_option = "outs"

    def get_affected_files(self, instance) -> list:
        """Define the filename that is passed to dvc (used if tracked=True)"""
        # self.name is the name of the class attribute we use for this database
        return [instance.nwd / f"{self.name}.db"]

    def save(self, instance):
        """Save the values to file"""
        # we gather the actual values using __get__
        atoms = getattr(instance, self.name)
        # get the file name
        file = self.get_affected_files(instance)[0]
        # save the data to the file
        with ase.db.connect(file) as db:
            for atom in tqdm.tqdm(atoms, ncols=70, desc=f"Writing atoms to {file}"):
                db.write(atom, group=instance.name)

    def _get_value_from_file(self, instance):
        """Load data with ase.db.connect from file"""
        # get the file name
        file = self.get_affected_files(instance)[0]
        # load the data
        atoms = []
        with ase.db.connect(file) as db:
            for row in tqdm.tqdm(
                db.select(), ncols=70, desc=f"Loading atoms from {file}"
            ):
                atoms.append(row.toatoms())
        # return the data so it can be saved in __dict__
        return atoms

    def get_stage_add_argument(self, instance) -> list:
        """Get the dvc stage add argument for this field."""
        # get the file name
        file = self.get_affected_files(instance)[0]
        # return the dvc stage add argument
        return [("--outs", file)]

Now that we have defined our custom ZnTrackOption we can use it as follows.

In [23]:
class AtomsClass(zntrack.Node):
    atoms = Atoms()

    def run(self):
        self.atoms = [ase.Atoms("N2", positions=[[0, 0, -1], [0, 0, 1]])]

In [24]:
atoms_class = AtomsClass()
atoms_class.run()
atoms_class.save()

[NbConvertApp] Converting notebook 08_custom_zntrackoptions.ipynb to script
[NbConvertApp] Writing 2731 bytes to 08_custom_zntrackoptions.py
Writing atoms to nodes/AtomsClass/atoms.db: 100%|█| 1/1 [00:00<00:00, 


In [25]:
AtomsClass.from_rev().atoms

Loading atoms from nodes/AtomsClass/atoms.db: 2it [00:00, 4801.72it/s]


[Atoms(symbols='N2', pbc=False), Atoms(symbols='N2', pbc=False)]

In [26]:
temp_dir.cleanup()